In [140]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc read-aloud"><h2>--- Day 14: Space Stoichiometry ---</h2><p>As you approach the rings of Saturn, your ship's <em>low fuel</em> indicator turns on.  There isn't any fuel here, but the rings have plenty of raw material.  Perhaps your ship's <span title="Yes, the acronym is intentional.">Inter-Stellar Refinery Union</span> brand <em>nanofactory</em> can turn these raw materials into fuel.</p>
<p>You ask the nanofactory to produce a list of the <em>reactions</em> it can perform that are relevant to this process (your puzzle input). Every reaction turns some quantities of specific <em>input chemicals</em> into some quantity of an <em>output chemical</em>. Almost every <em>chemical</em> is produced by exactly one reaction; the only exception, <code>ORE</code>, is the raw material input to the entire process and is not produced by a reaction.</p>
<p>You just need to know how much <code><em>ORE</em></code> you'll need to collect before you can produce one unit of <code><em>FUEL</em></code>.</p>
<p>Each reaction gives specific quantities for its inputs and output; reactions cannot be partially run, so only whole integer multiples of these quantities can be used.  (It's okay to have leftover chemicals when you're done, though.) For example, the reaction <code>1 A, 2 B, 3 C =&gt; 2 D</code> means that exactly 2 units of chemical <code>D</code> can be produced by consuming exactly 1 <code>A</code>, 2 <code>B</code> and 3 <code>C</code>.  You can run the full reaction as many times as necessary; for example, you could produce 10 <code>D</code> by consuming 5 <code>A</code>, 10 <code>B</code>, and 15 <code>C</code>.</p>
<p>Suppose your nanofactory produces the following list of reactions:</p>
<pre><code>10 ORE =&gt; 10 A
1 ORE =&gt; 1 B
7 A, 1 B =&gt; 1 C
7 A, 1 C =&gt; 1 D
7 A, 1 D =&gt; 1 E
7 A, 1 E =&gt; 1 FUEL
</code></pre>
<p>The first two reactions use only <code>ORE</code> as inputs; they indicate that you can produce as much of chemical <code>A</code> as you want (in increments of 10 units, each 10 costing 10 <code>ORE</code>) and as much of chemical <code>B</code> as you want (each costing 1 <code>ORE</code>).  To produce 1 <code>FUEL</code>, a total of <em>31</em> <code>ORE</code> is required: 1 <code>ORE</code> to produce 1 <code>B</code>, then 30 more <code>ORE</code> to produce the 7 + 7 + 7 + 7 = 28 <code>A</code> (with 2 extra <code>A</code> wasted) required in the reactions to convert the <code>B</code> into <code>C</code>, <code>C</code> into <code>D</code>, <code>D</code> into <code>E</code>, and finally <code>E</code> into <code>FUEL</code>. (30 <code>A</code> is produced because its reaction requires that it is created in increments of 10.)</p>
<p>Or, suppose you have the following list of reactions:</p>
<pre><code>9 ORE =&gt; 2 A
8 ORE =&gt; 3 B
7 ORE =&gt; 5 C
3 A, 4 B =&gt; 1 AB
5 B, 7 C =&gt; 1 BC
4 C, 1 A =&gt; 1 CA
2 AB, 3 BC, 4 CA =&gt; 1 FUEL
</code></pre>
<p>The above list of reactions requires <em>165</em> <code>ORE</code> to produce 1 <code>FUEL</code>:</p>
<ul>
<li>Consume 45 <code>ORE</code> to produce 10 <code>A</code>.</li>
<li>Consume 64 <code>ORE</code> to produce 24 <code>B</code>.</li>
<li>Consume 56 <code>ORE</code> to produce 40 <code>C</code>.</li>
<li>Consume 6 <code>A</code>, 8 <code>B</code> to produce 2 <code>AB</code>.</li>
<li>Consume 15 <code>B</code>, 21 <code>C</code> to produce 3 <code>BC</code>.</li>
<li>Consume 16 <code>C</code>, 4 <code>A</code> to produce 4 <code>CA</code>.</li>
<li>Consume 2 <code>AB</code>, 3 <code>BC</code>, 4 <code>CA</code> to produce 1 <code>FUEL</code>.</li>
</ul>
<p>Here are some larger examples:</p>
<ul>
<li><p><em>13312</em> <code>ORE</code> for 1 <code>FUEL</code>:</p>
<pre><code>157 ORE =&gt; 5 NZVS
165 ORE =&gt; 6 DCFZ
44 XJWVT, 5 KHKGT, 1 QDVJ, 29 NZVS, 9 GPVTF, 48 HKGWZ =&gt; 1 FUEL
12 HKGWZ, 1 GPVTF, 8 PSHF =&gt; 9 QDVJ
179 ORE =&gt; 7 PSHF
177 ORE =&gt; 5 HKGWZ
7 DCFZ, 7 PSHF =&gt; 2 XJWVT
165 ORE =&gt; 2 GPVTF
3 DCFZ, 7 NZVS, 5 HKGWZ, 10 PSHF =&gt; 8 KHKGT
</code></pre></li>
<li><p><em>180697</em> <code>ORE</code> for 1 <code>FUEL</code>:</p>
<pre><code>2 VPVL, 7 FWMGM, 2 CXFTF, 11 MNCFX =&gt; 1 STKFG
17 NVRVD, 3 JNWZP =&gt; 8 VPVL
53 STKFG, 6 MNCFX, 46 VJHF, 81 HVMC, 68 CXFTF, 25 GNMV =&gt; 1 FUEL
22 VJHF, 37 MNCFX =&gt; 5 FWMGM
139 ORE =&gt; 4 NVRVD
144 ORE =&gt; 7 JNWZP
5 MNCFX, 7 RFSQX, 2 FWMGM, 2 VPVL, 19 CXFTF =&gt; 3 HVMC
5 VJHF, 7 MNCFX, 9 VPVL, 37 CXFTF =&gt; 6 GNMV
145 ORE =&gt; 6 MNCFX
1 NVRVD =&gt; 8 CXFTF
1 VJHF, 6 MNCFX =&gt; 4 RFSQX
176 ORE =&gt; 6 VJHF
</code></pre></li>
<li><p><em>2210736</em> <code>ORE</code> for 1 <code>FUEL</code>:</p>
<pre><code>171 ORE =&gt; 8 CNZTR
7 ZLQW, 3 BMBT, 9 XCVML, 26 XMNCP, 1 WPTQ, 2 MZWV, 1 RJRHP =&gt; 4 PLWSL
114 ORE =&gt; 4 BHXH
14 VRPVC =&gt; 6 BMBT
6 BHXH, 18 KTJDG, 12 WPTQ, 7 PLWSL, 31 FHTLT, 37 ZDVW =&gt; 1 FUEL
6 WPTQ, 2 BMBT, 8 ZLQW, 18 KTJDG, 1 XMNCP, 6 MZWV, 1 RJRHP =&gt; 6 FHTLT
15 XDBXC, 2 LTCX, 1 VRPVC =&gt; 6 ZLQW
13 WPTQ, 10 LTCX, 3 RJRHP, 14 XMNCP, 2 MZWV, 1 ZLQW =&gt; 1 ZDVW
5 BMBT =&gt; 4 WPTQ
189 ORE =&gt; 9 KTJDG
1 MZWV, 17 XDBXC, 3 XCVML =&gt; 2 XMNCP
12 VRPVC, 27 CNZTR =&gt; 2 XDBXC
15 KTJDG, 12 BHXH =&gt; 5 XCVML
3 BHXH, 2 VRPVC =&gt; 7 MZWV
121 ORE =&gt; 7 VRPVC
7 XCVML =&gt; 6 RJRHP
5 BHXH, 4 VRPVC =&gt; 5 LTCX
</code></pre></li>
</ul>
<p>Given the list of reactions in your puzzle input, <em>what is the minimum amount of <code>ORE</code> required to produce exactly 1 <code>FUEL</code>?</em></p>
</article>


In [148]:
from itertools import batched

from more_itertools import one


tests = [
    {
        "name": "Example 1",
        "reactions": """
            10 ORE => 10 A
            1 ORE => 1 B
            7 A, 1 B => 1 C
            7 A, 1 C => 1 D
            7 A, 1 D => 1 E
            7 A, 1 E => 1 FUEL
        """,
        "expected": 31,
    },
    {
        "name": "Example 2",
        "reactions": """
            9 ORE => 2 A
            8 ORE => 3 B
            7 ORE => 5 C
            3 A, 4 B => 1 AB
            5 B, 7 C => 1 BC
            4 C, 1 A => 1 CA
            2 AB, 3 BC, 4 CA => 1 FUEL
        """,
        "expected": 165,
    },
    {
        "name": "Example ",
        "reactions": """
            157 ORE => 5 NZVS
            165 ORE => 6 DCFZ
            44 XJWVT, 5 KHKGT, 1 QDVJ, 29 NZVS, 9 GPVTF, 48 HKGWZ => 1 FUEL
            12 HKGWZ, 1 GPVTF, 8 PSHF => 9 QDVJ
            179 ORE => 7 PSHF
            177 ORE => 5 HKGWZ
            7 DCFZ, 7 PSHF => 2 XJWVT
            165 ORE => 2 GPVTF
            3 DCFZ, 7 NZVS, 5 HKGWZ, 10 PSHF => 8 KHKGT
        """,
        "expected": 13312,
    },
    {
        "name": "Example 4",
        "reactions": """
            2 VPVL, 7 FWMGM, 2 CXFTF, 11 MNCFX => 1 STKFG
            17 NVRVD, 3 JNWZP => 8 VPVL
            53 STKFG, 6 MNCFX, 46 VJHF, 81 HVMC, 68 CXFTF, 25 GNMV => 1 FUEL
            22 VJHF, 37 MNCFX => 5 FWMGM
            139 ORE => 4 NVRVD
            144 ORE => 7 JNWZP
            5 MNCFX, 7 RFSQX, 2 FWMGM, 2 VPVL, 19 CXFTF => 3 HVMC
            5 VJHF, 7 MNCFX, 9 VPVL, 37 CXFTF => 6 GNMV
            145 ORE => 6 MNCFX
            1 NVRVD => 8 CXFTF
            1 VJHF, 6 MNCFX => 4 RFSQX
            176 ORE => 6 VJHF
        """,
        "expected": 180697,
    },
    {
        "name": "Example 5",
        "reactions": """
            171 ORE => 8 CNZTR
            7 ZLQW, 3 BMBT, 9 XCVML, 26 XMNCP, 1 WPTQ, 2 MZWV, 1 RJRHP => 4 PLWSL
            114 ORE => 4 BHXH
            14 VRPVC => 6 BMBT
            6 BHXH, 18 KTJDG, 12 WPTQ, 7 PLWSL, 31 FHTLT, 37 ZDVW => 1 FUEL
            6 WPTQ, 2 BMBT, 8 ZLQW, 18 KTJDG, 1 XMNCP, 6 MZWV, 1 RJRHP => 6 FHTLT
            15 XDBXC, 2 LTCX, 1 VRPVC => 6 ZLQW
            13 WPTQ, 10 LTCX, 3 RJRHP, 14 XMNCP, 2 MZWV, 1 ZLQW => 1 ZDVW
            5 BMBT => 4 WPTQ
            189 ORE => 9 KTJDG
            1 MZWV, 17 XDBXC, 3 XCVML => 2 XMNCP
            12 VRPVC, 27 CNZTR => 2 XDBXC
            15 KTJDG, 12 BHXH => 5 XCVML
            3 BHXH, 2 VRPVC => 7 MZWV
            121 ORE => 7 VRPVC
            7 XCVML => 6 RJRHP
            5 BHXH, 4 VRPVC => 5 LTCX
        """,
        "expected": 2210736,
    },
]


class Graph:
    def __init__(self, s: str) -> None:
        self.graph = self._parse(s)

    def _parse(self, reactions: str):
        graph = {"ORE": {"batch": 1, "tos": {}}}

        for line in reactions.strip().splitlines():
            fr_s, to_s = line.split(" => ")
            batch_size, to_name = one(
                (int(q), n) for q, n in batched(to_s.split(" "), 2)
            )

            if to_name not in graph:
                graph[to_name] = {"batch": batch_size, "tos": {}}
            else:
                graph[to_name]["batch"] = batch_size

            for l in fr_s.strip().split(", "):
                for quantity, name in batched(l.split(" "), 2):
                    if name not in graph:
                        graph[name] = {"batch": batch_size, "tos": {}}
                    graph[name]["tos"][to_name] = int(quantity)

        return graph

    def ORE_required(self, fuel_need=1) -> int:
        def dfs(node) -> int:
            if node == "FUEL":
                return fuel_need

            batch = self.graph[node]["batch"]
            total_quantity = 0

            for to, quantity in self.graph[node]["tos"].items():
                need = dfs(to)
                total_quantity += quantity * need / self.graph[to]["batch"]

            total_quantity, mod = divmod(total_quantity, batch)
            total_quantity *= batch
            if mod > 0:
                total_quantity += batch
            return int(total_quantity)

        return dfs("ORE")

    def max_fuel(self, total_ore=1000000000000) -> int:
        low, high = 0, total_ore
        while low < high:
            mid = (low + high) // 2
            ore_need = self.ORE_required(mid)
            if ore_need < total_ore:
                low = mid + 1
            else:
                high = mid

        while ore_need > total_ore:
            mid -= 1
            ore_need = self.ORE_required(mid)

        return mid

    def __repr__(self) -> str:
        s = "Graph {\n"
        for to, vals in self.graph.items():
            quantity = vals.get("batch", 1)
            s += f"\t{to}: {quantity} {vals.get('tos', {})}\n"
        return s + "\n}"

    def print(self) -> str:
        G = nx.DiGraph()
        G.add_nodes_from(self.graph.keys())

        pos = nx.spiral_layout(G, equidistant=True)

        options = {"node_size": 2000}
        nx.draw_networkx_nodes(G, pos, **options)

        labels = {node: f"{node}/{self.graph[node]['batch']}" for node in G.nodes}
        nx.draw_networkx_labels(G, pos, labels)

        for fr in self.graph.keys():
            for to, w in self.graph[fr].get("tos", {}).items():
                G.add_edge(fr, to, weight=w)

        edge_labels = nx.get_edge_attributes(G, "weight")
        nx.draw_networkx_edges(
            G,
            pos,
            width=2,
            edge_color="r",
            arrows=True,
            arrowsize=20,
            arrowstyle="-|>",
        )
        nx.draw_networkx_edge_labels(G, pos, edge_labels)

        ax = plt.gca()
        ax.margins(0.08)
        plt.axis("off")
        plt.tight_layout()
        plt.show()

    @classmethod
    def part_I(cls, reactions: str) -> int:
        return cls(reactions).ORE_required()

    @classmethod
    def part_II(cls, reactions: str) -> int:
        return cls(reactions).max_fuel()


run_tests_params(Graph.part_I, tests)

# Graph(tests[0]["reactions"]).print()


Test Example 1 passed, for part_I.
Test Example 2 passed, for part_I.
Test Example  passed, for part_I.
Test Example 4 passed, for part_I.
Test Example 5 passed, for part_I.
Success


In [149]:
with open("../input/day14.txt") as f:
    puzzle = f.read()

print(f"Part I: {Graph.part_I(puzzle)}")
# Graph(puzzle).print()

Part I: 485720


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>485720</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>After collecting <code>ORE</code> for a while, you check your cargo hold: <em>1 trillion</em> (<em>1000000000000</em>) units of <code>ORE</code>.</p>
<p><em>With that much ore</em>, given the examples above:</p>
<ul>
<li>The 13312 <code>ORE</code>-per-<code>FUEL</code> example could produce <em>82892753</em> <code>FUEL</code>.</li>
<li>The 180697 <code>ORE</code>-per-<code>FUEL</code> example could produce <em>5586022</em> <code>FUEL</code>.</li>
<li>The 2210736 <code>ORE</code>-per-<code>FUEL</code> example could produce <em>460664</em> <code>FUEL</code>.</li>
</ul>
<p>Given 1 trillion <code>ORE</code>, <em>what is the maximum amount of <code>FUEL</code> you can produce?</em></p>
</article>

</main>


In [150]:
tests_part_II = [
    {
        "name": "Example 3",
        "reactions": """
            157 ORE => 5 NZVS
            165 ORE => 6 DCFZ
            44 XJWVT, 5 KHKGT, 1 QDVJ, 29 NZVS, 9 GPVTF, 48 HKGWZ => 1 FUEL
            12 HKGWZ, 1 GPVTF, 8 PSHF => 9 QDVJ
            179 ORE => 7 PSHF
            177 ORE => 5 HKGWZ
            7 DCFZ, 7 PSHF => 2 XJWVT
            165 ORE => 2 GPVTF
            3 DCFZ, 7 NZVS, 5 HKGWZ, 10 PSHF => 8 KHKGT
        """,
        "expected": 82892753,
    },
    {
        "name": "Example 4",
        "reactions": """
            2 VPVL, 7 FWMGM, 2 CXFTF, 11 MNCFX => 1 STKFG
            17 NVRVD, 3 JNWZP => 8 VPVL
            53 STKFG, 6 MNCFX, 46 VJHF, 81 HVMC, 68 CXFTF, 25 GNMV => 1 FUEL
            22 VJHF, 37 MNCFX => 5 FWMGM
            139 ORE => 4 NVRVD
            144 ORE => 7 JNWZP
            5 MNCFX, 7 RFSQX, 2 FWMGM, 2 VPVL, 19 CXFTF => 3 HVMC
            5 VJHF, 7 MNCFX, 9 VPVL, 37 CXFTF => 6 GNMV
            145 ORE => 6 MNCFX
            1 NVRVD => 8 CXFTF
            1 VJHF, 6 MNCFX => 4 RFSQX
            176 ORE => 6 VJHF
        """,
        "expected": 5586022,
    },
    {
        "name": "Example 5",
        "reactions": """
            171 ORE => 8 CNZTR
            7 ZLQW, 3 BMBT, 9 XCVML, 26 XMNCP, 1 WPTQ, 2 MZWV, 1 RJRHP => 4 PLWSL
            114 ORE => 4 BHXH
            14 VRPVC => 6 BMBT
            6 BHXH, 18 KTJDG, 12 WPTQ, 7 PLWSL, 31 FHTLT, 37 ZDVW => 1 FUEL
            6 WPTQ, 2 BMBT, 8 ZLQW, 18 KTJDG, 1 XMNCP, 6 MZWV, 1 RJRHP => 6 FHTLT
            15 XDBXC, 2 LTCX, 1 VRPVC => 6 ZLQW
            13 WPTQ, 10 LTCX, 3 RJRHP, 14 XMNCP, 2 MZWV, 1 ZLQW => 1 ZDVW
            5 BMBT => 4 WPTQ
            189 ORE => 9 KTJDG
            1 MZWV, 17 XDBXC, 3 XCVML => 2 XMNCP
            12 VRPVC, 27 CNZTR => 2 XDBXC
            15 KTJDG, 12 BHXH => 5 XCVML
            3 BHXH, 2 VRPVC => 7 MZWV
            121 ORE => 7 VRPVC
            7 XCVML => 6 RJRHP
            5 BHXH, 4 VRPVC => 5 LTCX
        """,
        "expected": 460664,
    },
]

run_tests_params(Graph.part_II, tests_part_II)


Test Example 3 passed, for part_II.
Test Example 4 passed, for part_II.
Test Example 5 passed, for part_II.
Success


In [151]:
print(f"Part II: {Graph.part_II(puzzle)}")

Part II: 3848998


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>3848998</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
